<a href="https://colab.research.google.com/github/crypticalmass/SystemPrompts_research/blob/main/Grounding_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Gemini API: Getting started with information grounding for Gemini models

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Grounding.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

In this notebook you will learn how to use information grounding with [Gemini models](https://ai.google.dev/gemini-api/docs/models/).

Information grounding is the process of connecting these models to specific, verifiable information sources to enhance the accuracy, relevance, and factual correctness of their responses. While LLMs are trained on vast amounts of data, this knowledge can be general, outdated, or lack specific context for particular tasks or domains. Grounding helps to bridge this gap by providing the LLM with access to curated, up-to-date information.

Here you will experiment with:
- Grounding information using <a href="#search_grounding">Google Search grounding</a>
- Adding <a href="#yt_links">YouTube links</a> to gather context information to your prompt
- Using <a href="#url_context">URL context</a> to include website, pdf or image URLs as context to your prompt

## Set up the SDK and the client

### Install SDK

This guide uses the [`google-genai`](https://pypi.org/project/google-genai) Python SDK to connect to the Gemini models.

In [ ]:
# Grounding with Google Maps was introduced in 1.43
%pip install -q -U "google-genai>=1.43.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.9/721.9 kB 7.4 MB/s eta 0:00:00


### Set up your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the [Authentication ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](../quickstarts/Authentication.ipynb) quickstart for an example.

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

### Select model and initialize SDK client

Select the model you want to use in this guide, either by selecting one in the list or writing it down. Keep in mind that some models, like the 2.5 ones are thinking models and thus take slightly more time to respond (cf. [thinking notebook](./Get_started_thinking.ipynb) for more details and in particular learn how to switch the thiking off).

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

MODEL_ID = "gemini-3-flash-preview" # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-2.5-flash-preview", "gemini-3-pro-preview"] {"allow-input":true, isTemplate: true}

<a name="search_grounding"></a>

## Use Google Search grounding

Google Search grounding is particularly useful for queries that require current information or external knowledge. Using Google Search, Gemini can access nearly real-time information and better responses.

To enable Google Search, simply add the `google_search` tool in the `generate_content`'s `config` that way:
```
    config={
      "tools": [
        {
          "google_search": {}
        }
      ]
    },
```

In [ ]:
from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What was the latest Indian Premier League match and who won?",
    config={"tools": [{"google_search": {}}]},
)

# print the response
display(Markdown(f"**Response**:\n {response.text}"))
# print the search details
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
# urls used for grounding
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

**Response**:
 The latest match of the Indian Premier League (IPL) was the **2025 Final**, which took place on **June 3, 2025**.

The **Royal Challengers Bengaluru (RCB)** won the match, defeating the **Punjab Kings (PBKS)** by **6 runs** to secure their maiden IPL title. 

### **Match Summary:**
*   **Result:** Royal Challengers Bengaluru won by 6 runs.
*   **Scores:**
    *   **RCB:** 190/9 (20 overs)
    *   **PBKS:** 184/7 (20 overs)
*   **Venue:** Narendra Modi Stadium, Ahmedabad (rescheduled from Eden Gardens).
*   **Key Performers:** RCB's victory was anchored by a strong total of 190, which they defended successfully despite a late charge from PBKS's Shashank Singh (61* off 30 balls). Bowling highlights for RCB included Krunal Pandya and Bhuvneshwar Kumar, who both took two wickets each.
*   **Significance:** This victory marked RCB's first-ever IPL trophy in the tournament's 18-year history.

Prior to the 2025 season, the **2024 IPL Final** was won by the **Kolkata Knight Riders (KKR)**, who defeated the Sunrisers Hyderabad (SRH) by 8 wickets on May 26, 2024. As of February 2026, the 2026 IPL season has not yet begun, as the tournament typically commences in late March or April.

Search Query: ['latest IPL match result 2024 winner', 'who won the last IPL match']
Search Pages: jagranjosh.com, olympics.com, wikipedia.org, youtube.com, wikipedia.org


You can see that running the same prompt without search grounding gives you outdated information:

In [ ]:
from IPython.display import Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="What was the latest Indian Premier League match and who won?",
)

# print the response
display(Markdown(response.text))

The latest match in the Indian Premier League (IPL 2024) was **Qualifier 2**, which took place on May 24, 2024.

**Sunrisers Hyderabad (SRH)** won the match, defeating the Rajasthan Royals (RR) by **36 runs**.

**Match Summary:**
*   **SRH Score:** 175/9 (20 overs)
*   **RR Score:** 139/7 (20 overs)
*   **Venue:** MA Chidambaram Stadium, Chennai
*   **Key Performers:** Heinrich Klaasen scored a vital 50 for SRH, while Shahbaz Ahmed (3/23) and Abhishek Sharma (2/24) dominated with the ball to restrict Rajasthan.

With this win, **Sunrisers Hyderabad** have qualified for the Final, where they will face the **Kolkata Knight Riders (KKR)** on Sunday, May 26.

For more examples, please refer to the [dedicated notebook ![image](https://storage.googleapis.com/generativeai-downloads/images/colab_icon16.png)](./Search_Grounding.ipynb).

<a name="maps_grounding"></a>

<a name="yt_links"></a>

## Grounding with YouTube links

You can directly include a public YouTube URL in your prompt. The Gemini models will then process the video content to perform tasks like summarization and answering questions about the content.

This capability leverages Gemini's multimodal understanding, allowing it to analyze and interpret video data alongside any text prompts provided.

You do need to explicitly declare the video URL you want the model to process as part of the contents of the request using a `FileData` part. Here a simple interaction where you ask the model to summarize a YouTube video:

In [ ]:
yt_link = "https://www.youtube.com/watch?v=XV1kOFo1C8M"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=types.Content(
        parts=[
            types.Part(text="Summarize this video."),
            types.Part(file_data=types.FileData(file_uri=yt_link)),
        ]
    ),
)

Markdown(response.text)

Ju-yeong Ji, from Google DeepMind's AI Developer Relations, presents Gemma’s capabilities in the world of chess. He emphasizes that while AI chess engines like AlphaZero excel at move calculation, Gemma offers a new dimension through its language-processing strengths.

Gemma's key features in chess include:

*   **Move Analysis:** It can translate technical chess engine outputs into easy-to-understand explanations, detailing why a move is strategic, what its goals are, and what potential risks are involved.
*   **Storytelling:** Gemma can narrate chess matches by incorporating context like players, tournaments, and historical significance, making games more engaging than just a series of moves.
*   **Educational Support:** It serves as a personalized tutor, explaining chess concepts and strategies in multiple languages and adjusting its level to the player's expertise.
*   **Integration with Analytical Engines:** Gemma can be combined with chess engines to provide both the best possible moves and clear, human-readable explanations of the underlying strategy.

Overall, Gemma aims to make chess more accessible and engaging by bridging the gap between technical analysis and human understanding.

But you can also use the link as the source of truth for your request. In this example, you will first ask how Gemma models can help on chess games:

In [ ]:
yt_link = "https://www.youtube.com/watch?v=XV1kOFo1C8M"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=types.Content(
        parts=[
            types.Part(
                text="In 2 paragraph, how Gemma models can help on chess games?"
            ),
            types.Part(file_data=types.FileData(file_uri=yt_link)),
        ]
    ),
)

Markdown(response.text)

Gemma models significantly enhance the chess experience by bridging the gap between technical analysis and human understanding. While traditional chess engines excel at calculating optimal moves, Gemma can interpret these results and explain the strategic reasoning behind them in plain language. It can analyze complex game sequences to pinpoint key tactical moments and articulate the "why" behind a particular move, such as controlling the center or setting up a future attack. Furthermore, Gemma can transform chess games into compelling narratives, weaving together moves, player backgrounds, and tournament contexts to bring the history and drama of the game to life.

As an educational tool, Gemma acts as a personalized chess tutor available 24/7 in multiple languages. It can explain fundamental and advanced concepts, from opening theories like the Sicilian Defense to specific tactical ideas like passed pawns, tailoring its explanations to the user’s skill level. By integrating with analytical engines, Gemma provides a comprehensive learning experience—it can retrieve the best possible move and then generate an intuitive explanation of its advantages. This combination of deep calculation and linguistic capability makes chess more accessible and engaging for beginners and advanced players alike.

Now your answer is more insightful for the topic you want, using the knowledge shared on the video and not necessarily available on the model knowledge.

<a name="url_context"></a>

## Grounding information using URL context

The URL Context tool empowers Gemini models to directly access and process content from specific web page URLs you provide within your API requests. This is incredibly interesting because it allows your applications to dynamically interact with live web information without needing you to manually pre-process and feed that content to the model.

URL Context is effective because it allows the models to base its responses and analysis directly on the content of the designated web pages. Instead of relying solely on its general training data or broad web searches (which are also valuable grounding tools), URL Context anchors the model's understanding to the specific information present at those URLs.

### Process website URLs

If you want Gemini to specifically ground its answers thanks to the content of a specific website, just add the urls in your prompt and enable the tool by adding it to your config:
```
config = {
  "tools": [
    {
      "url_context": {}
    }
  ],
}
```

You can add up to 20 links in your prompt.

In [ ]:
prompt = """
  Based on https://ai.google.dev/gemini-api/docs/models, what are the key
  differences between Gemini 1.5, Gemini 2.0, Gemini 2.5 and Gemini 3 models?
  Create a markdown table comparing the differences.
"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text))

Based on the current documentation at [ai.google.dev/gemini-api/docs/models](https://ai.google.dev/gemini-api/docs/models) (as of February 2026), the Gemini model lineup has evolved through several generations, focusing on intelligence, reasoning, and efficiency.

The following table summarizes the key differences between Gemini 1.5, 2.0, 2.5, and 3.

### Comparison Table: Gemini Model Generations

| Feature | Gemini 1.5 (Legacy) | Gemini 2.0 (Deprecated) | Gemini 2.5 (Current Performance) | Gemini 3 (Frontier Intelligence) |
| :--- | :--- | :--- | :--- | :--- |
| **Core Value** | Original long-context workhorse. | Next-gen features & native tool use. | Best price-performance & advanced thinking. | Most intelligent, agentic, & vibe-coding model. |
| **Top Model** | Pro 1.5 | Flash 2.0 | Pro 2.5 | Pro 3 |
| **Max Context Window** | 1M - 2M tokens | 1M tokens | 1M tokens | 1M+ tokens |
| **Max Output Tokens** | ~8K tokens | 8,192 tokens | 65,536 tokens | 65,536 tokens |
| **Thinking Support** | Limited / Basic | Experimental | **Full Support** (Pro & Flash) | **Optimized** (State-of-the-art) |
| **Capabilities** | Multimodal (Text, Image, Audio, Video) | Native Tool Use, Superior Speed | Agentic workflows, High-volume processing | Richer visuals, Deeper interactivity, Frontier reasoning |
| **Status** | Legacy / Retired | Deprecated (Shutdown Mar 2026) | **Stable** / Production Ready | **Preview** / Latest Intelligence |

---

### Key Differences Explained

1.  **Gemini 3 (Frontier Intelligence):**
    *   **Agentic Power:** Specifically designed for complex agentic workflows and "vibe-coding" (highly interactive, intuitive coding and development).
    *   **Intelligence:** Currently ranked as the most intelligent model for multimodal understanding and reasoning.
    *   **Enhanced Output:** Offers a massive increase in output token limits (up to 64K+) compared to earlier generations, allowing for much longer generated responses.

2.  **Gemini 2.5 (High Performance & Reasoning):**
    *   **Price-Performance:** The "Flash" version is optimized for large-scale processing and low latency while maintaining "Thinking" capabilities.
    *   **Thinking Model:** Gemini 2.5 Pro is the standard for complex reasoning in code, math, and STEM, effectively replacing the niche formerly held by 1.5 Pro.
    *   **Native Audio:** Includes specialized versions (like Flash-Native-Audio) for real-time audio generation and understanding via the Live API.

3.  **Gemini 2.0 (Second Generation - Deprecated):**
    *   **Transitionary:** Introduced native tool use and improved speed over the 1.0/1.5 series.
    *   **Legacy Limits:** Limited by lower output token caps (8K) and experimental "Thinking" features compared to the robust implementation in 2.5 and 3.

4.  **Gemini 1.5 (The Context Pioneer):**
    *   **Legacy Status:** While it pioneered the 1M+ token context window, it is now considered legacy. Its reasoning and speed have been surpassed by the Gemini 2.5 Flash and Pro models, which offer similar context windows with significantly better intelligence and efficiency.

You can see the status of the retrival using `url_context_metadata`:

In [ ]:
# get URLs retrieved for context
print(response.candidates[0].url_context_metadata)

url_metadata=[UrlMetadata(
  retrieved_url='https://ai.google.dev/gemini-api/docs/models',
  url_retrieval_status=<UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS: 'URL_RETRIEVAL_STATUS_SUCCESS'>
), UrlMetadata(
  retrieved_url='https://ai.google.dev/gemini-api/docs/models',
  url_retrieval_status=<UrlRetrievalStatus.URL_RETRIEVAL_STATUS_SUCCESS: 'URL_RETRIEVAL_STATUS_SUCCESS'>
)]


### Add PDFs by URL

Gemini can also process PDFs from an URL. Here's an example:

In [ ]:
prompt = """
  Can you give me an overview of the content of this pdf?
  https://abc.xyz/assets/cc/27/3ada14014efbadd7a58472f1f3f4/2025q2-alphabet-earnings-release.pdf

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text.replace("$", "\$")))

<>:15: SyntaxWarning: invalid escape sequence '\$'
<>:15: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-3794376347.py:15: SyntaxWarning: invalid escape sequence '\$'
  display(Markdown(response.text.replace("$", "\$")))


Alphabet's Q2 2025 earnings release reports significant financial growth, driven primarily by strong performance in Google Search and the expanding Google Cloud division. The company recorded substantial year-over-year increases in both total revenue and net income, reflecting robust advertiser demand and AI-driven product enhancements. Leadership highlighted ongoing investments in artificial intelligence infrastructure as a key factor in improving user experience and operational efficiency. Additionally, the report outlines capital allocation strategies, including share repurchases and the continued scaling of YouTube’s subscription services.

In [ ]:
prompt = """
  Can you explain Figure 1 in this PDF file
  https://arxiv.org/pdf/1706.03762

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text.replace("$", "\$")))

<>:15: SyntaxWarning: invalid escape sequence '\$'
<>:15: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-4075067263.py:15: SyntaxWarning: invalid escape sequence '\$'
  display(Markdown(response.text.replace("$", "\$")))


Figure 1 in the paper **"Attention Is All You Need"** represents the **Transformer model architecture**. It is the blueprint for the modern AI models we use today (like GPT-4 and Claude).

The diagram is split into two main sections: the **Encoder** (on the left) and the **Decoder** (on the right). Here is a detailed breakdown of what is happening in each part:

### 1. The Encoder (Left Side)
The job of the encoder is to process the input text (like an English sentence) and turn it into a mathematical representation that captures the meaning and context of every word.
*   **Input Embedding & Positional Encoding:** Words are converted into numbers (embeddings). Since Transformers don't process words one-by-one (like older RNNs), they add "Positional Encodings" to give the model a sense of where each word is located in the sentence.
*   **Multi-Head Attention:** This is the "secret sauce." It allows the model to look at every other word in the sentence simultaneously to understand context. For example, in the sentence *"The bank of the river,"* the word "bank" attends to "river" to know it's not a financial institution.
*   **Feed Forward:** A simple neural network that processes the information further.
*   **Nx Block:** The "Nx" indicates that this entire block is stacked 6 times to create a deep, complex understanding of the text.

### 2. The Decoder (Right Side)
The decoder’s job is to take the encoder's representation and generate the output (like a French translation), one word at a time.
*   **Masked Multi-Head Attention:** Similar to the encoder, but "masked" so that when the model is predicting the next word, it can't "cheat" by looking at the words that come after it.
*   **Encoder-Decoder Attention:** This is a crucial bridge. It allows the decoder to "look back" at the original input sentence (from the encoder) to make sure the translation stays accurate to the source.
*   **Linear & Softmax:** At the very top, the model turns its internal math back into a probability distribution over its entire vocabulary to pick the most likely next word.

### 3. Key Symbols in the Diagram
*   **Add & Norm:** These are "Residual Connections" followed by "Layer Normalization." They are technical tricks that allow the model to be very deep without the math "breaking" or becoming unstable during training.
*   **Arrows:** The arrows show the flow of data. Notice the arrow going from the top of the Encoder into the middle of the Decoder—this represents the decoder "paying attention" to the input sentence while it generates the output.

### Summary
In short, Figure 1 shows a system that **reads** a whole sentence at once (Encoder), **remembers** the context of every word (Self-Attention), and then **writes** a response (Decoder) by constantly referring back to what it read.

### Add images by URL

Gemini can also process images from an URL. Here's an example:

In [ ]:
prompt = """
  Can you help me name of the numbered parts of that instrument, in French?
  https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Trombone.svg/960px-Trombone.svg.png

"""

config = {
    "tools": [{"url_context": {}}],
}

response = client.models.generate_content(
    contents=[prompt], model=MODEL_ID, config=config
)

display(Markdown(response.text))

Voici les noms des différentes parties du trombone numérotées sur l'image, en français :

1. **Coulisse d'accord** (permet d'ajuster la justesse générale de l'instrument).
2. **Embouchure** (la pièce où le musicien pose ses lèvres).
3. **Pavillon** (la partie évasée d'où sort le son).
4. **Clé d'eau** (ou clé de purge, pour évacuer la condensation).
5. **Coulisse (mobile)** (la partie que l'on fait glisser pour changer les notes).
6. **Entretoise de la coulisse** (la barre transversale que le musicien tient pour manipuler la coulisse).
7. **Entretoise du pavillon** (barre de renfort sur la section du pavillon).
8. **Verrou de la coulisse** (petite bague qui permet de bloquer la coulisse lorsqu'on ne joue pas).

## Mix Search grounding and URL context

The different tools can also be use in conjunction by adding them both to the config. It's a good way to steer Gemini in the right direction and then let it do its magic using search grounding.

In [ ]:
prompt = """
  Can you give me an overview of the content of this pdf?
  https://abc.xyz/assets/cc/27/3ada14014efbadd7a58472f1f3f4/2025q2-alphabet-earnings-release.pdf
  Search on the web for the reaction of the main financial analysts, what's the trend?
"""

config = {
  "tools": [
      {"url_context": {}},
      {"google_search": {}}
  ],
}

response = client.models.generate_content(
  contents=[prompt],
  model=MODEL_ID,
  config=config
)

display(Markdown(response.text.replace('$','\$')))
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

<>:20: SyntaxWarning: invalid escape sequence '\$'
<>:20: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-110788399.py:20: SyntaxWarning: invalid escape sequence '\$'
  display(Markdown(response.text.replace('$','\$')))


The Alphabet Q2 2025 earnings release (dated July 23, 2025) depicts a company in a state of "accelerated transformation," balancing robust core growth with massive, front-loaded investments in artificial intelligence infrastructure.

### **Overview of the Earnings Release Content**

The report highlights a "double beat" on both the top and bottom lines, driven primarily by a surge in Google Cloud and the resilience of Search advertising.

*   **Top-Line Financials:** 
    *   **Revenue:** \$96.43 billion, a **14% increase** year-over-year (YoY).
    *   **Net Income:** \$28.2 billion, up **19% YoY**.
    *   **EPS (Earnings Per Share):** \$2.31, significantly beating the analyst consensus of ~\$2.17.
*   **Segment Performance:**
    *   **Google Search & Other:** Delivered double-digit revenue growth (\$71.34B), dismissing fears that AI-led search competition would immediately erode its core business.
    *   **Google Cloud:** A standout performer with **32% growth** (reaching \$13.6B). Notably, its annual revenue run-rate surpassed **\$50 billion**, achieving record profitability as enterprise adoption of Gemini and AI infrastructure scaled.
    *   **YouTube:** Continued growth in YouTube ads and subscriptions, with management noting that **YouTube Shorts** now generates revenue per watch-hour comparable to traditional instream ads in the U.S.
*   **Strategic & Capital Allocation:**
    *   **CapEx Surge:** Alphabet raised its total 2025 capital expenditure outlook to approximately **\$85 billion** (up from previous estimates of ~\$75B), focused almost entirely on AI servers and data centers.
    *   **AI Integration:** The release emphasized the rollout of **Gemini 2.5** and the success of **AI Overviews**, which now serves over 2 billion monthly users.

---

### **Financial Analyst Reaction & Market Trends**

The overall trend among Wall Street analysts is **"bullish but cautious regarding the ROI of spending."** While the operational results were stellar, the massive increase in CapEx sparked a debate over when these investments will translate into bottom-line returns.

#### **1. Main Analyst Reactions**
*   **Goldman Sachs:** Raised its price target to **\$234** (from \$225), citing the "accelerating business fundamentals" and Alphabet’s leadership in AI and cloud computing. They view the high CapEx as a necessary move to secure long-term dominance.
*   **BMO Capital:** Reiterated an **Outperform** rating with a **\$200** target, highlighting that Google’s AI advancements (like Rich Communication Services and Gemini) are successfully defending its ad-revenue moat.
*   **Morgan Stanley:** Maintained a target of **\$185**, focusing on the "improvement in ad relevance" and the potential of Cloud to become a primary earnings driver alongside Search.
*   **Seeking Alpha / Independent Analysts:** Noted a slight drop in Google's total search market share (down to 89% from 92% per StatCounter) due to ChatGPT and Perplexity, but characterized the company as "quietly dominating" because the total market for search is still expanding.

#### **2. Key Trends Identified**
*   **The "Cloud as the Second Engine":** Analysts have shifted their view of Alphabet from a one-trick pony (Ads) to a dual-engine company. The 32% growth in Cloud is seen as a signal that Alphabet is successfully capturing the enterprise AI "arms race."
*   **Search Resilience:** The "bear thesis" that AI chatbots would kill Google Search has largely been tabled for now. Analysts noted that "AI Overviews" actually appear to be increasing user engagement rather than cannibalizing clicks.
*   **Efficiency vs. Investment:** There is a growing trend of analysts watching the "Other Income" and "Operating Margins" (32.4%) closely. While Alphabet is growing, the cost of talent and hardware is rising, leading to a "show-me" story for 2026 regarding the actual profitability of these AI services.

**Summary Trend:** The stock is currently viewed as a **"Core AI Holding."** The market has accepted high spending as a prerequisite for survival, rewarding Alphabet for its ability to maintain high margins while rebuilding its entire tech stack for the AI era.